In [1]:
# %load ../env.py
%load_ext autoreload
%autoreload 2
%pylab inline
%matplotlib inline

import matplotlib.text as text
import pandas as pd
import numpy as np
import pylab as pl
import scipy as sp
import sys
import rpy2 
import os 

#Set environment variables


# Set up the local source files
TOP = '/'.join(os.getcwd().split('/')[:-2])+'/'
#TOP = "/share/home/ishah/ipynb/pb/HTTR_WF/"

LIB = TOP+'lib'
if not LIB in sys.path: 
    sys.path.insert(0,LIB)

os.environ['PYTHONPATH']=LIB


DAT_DIR = TOP + '/data/'
FIG_DIR = TOP + '/figs/'

if not os.path.exists(DAT_DIR): os.mkdir(DAT_DIR)
if not os.path.exists(FIG_DIR): os.mkdir(FIG_DIR)


from db.mongo import *

DB = openMongo(db='genra_dev_v4')

Populating the interactive namespace from numpy and matplotlib


In [3]:
DB.tox_fp.drop()

# Goal

Load toxicity information from ToxRefDB

1. ToxRefDB existing

`mongorestore -j20 -u ishah -p ishah -d genra_dev_v4 -c tox_fp tox5_fp.bson`

2. ToxRefDB v2




In [7]:
DB.collection_names()

[u'logs', u'tox_fp', u'compounds', u'chm_fp', u'bio_fp']

In [9]:
DB.tox_fp.find_one()

{u'_id': ObjectId('571a4d59b6597d78196d861c'),
 u'chemical_casrn': u'53-86-1',
 u'chemical_name': u'Indomethacin',
 u'dose_max': 20.0,
 u'dose_min': 20.0,
 u'dose_units': u'mg/kg/day',
 u'dsstox_cid': u'DTXCID50740',
 u'dsstox_sid': u'DTXSID9020740',
 u'name': u'Indomethacin',
 u'tox_fpn1': {u'ds': [u'ACU:Thymus',
   u'ACU:Mortality',
   u'ACU:Intestine Small',
   u'ACU:Testes',
   u'ACU:Ovary',
   u'ACU:Food Consumption',
   u'ACU:Adrenal Gland',
   u'ACU:Nose',
   u'ACU:Clinical Signs',
   u'ACU:Spleen',
   u'ACU:Eye',
   u'ACU:Body Weight',
   u'ACU:Lung',
   u'ACU:Water Consumption',
   u'ACU:Stomach',
   u'ACU:Intestine Large'],
  u'n': 16},
 u'tox_fpn2': {u'ds': [u'ACU:ReproductiveFemale:Ovary',
   u'ACU:ReproductiveMale:Testes',
   u'ACU:InLifeObservations:Clinical Signs',
   u'ACU:InLifeObservations:Water Consumption',
   u'ACU:PrimaryDigestive:Intestine Small',
   u'ACU:Integumentary:Nose',
   u'ACU:Lymphatic:Spleen',
   u'ACU:InLifeObservations:Food Consumption',
   u'ACU:InL

In [10]:
DB.tox_fp.find_one().keys()

[u'dose_min',
 u'tox_q2',
 u'chemical_casrn',
 u'dose_units',
 u'dose_max',
 u'tox_fpn1',
 u'tox_fpn2',
 u'chemical_name',
 u'dsstox_cid',
 u'tox_q1',
 u'dsstox_sid',
 u'_id',
 u'tox_fpp2',
 u'tox_fpp1',
 u'name']

## Let's look at new toxrefdb data 

In [2]:
DB.collection_names()

[u'logs',
 u'pred_gh',
 u'epa_categories',
 u'fp_stats',
 u'chemotypes',
 u'pred_expansion',
 u'tox1_fp',
 u'tox_fp',
 u'chm_fp',
 u'pred_is_v1',
 u'compound',
 u'bio_fp',
 u'tox21_fp',
 u'invitrodb',
 u'tox5_fp',
 u'acutetox',
 u'study_calls_2',
 u'pred_filter',
 u'physprop',
 u'toxrefdb2',
 u'study_calls_1']

In [2]:
DB.toxrefdb2.count()
DB.toxrefdb2.find(dict(bmds={'$exists':1})).count()

1084

631

In [3]:
X=DB.toxrefdb2.find_one(dict(preferred_name={'$regex':'Bisphenol.*','$options':'i'}, ))
#X=DB.toxrefdb2.find_one()
Y = pd.DataFrame(X['pods'])
Y.head()

,dose_level,effect_desc,endpoint_category,endpoint_target,endpoint_type,group_name,max_dose_level,mg_kg_day_value,pod_type,pod_unit,pod_value,qualifier
0,6,inflammation,systemic,liver,pathology microscopic,liver,6,500.000,loael,mg/kg/day,500.000,>
1,6,inflammation,systemic,liver,pathology microscopic,liver,6,500.000,noael,mg/kg/day,500.000,>=
2,1,"hepatocyte, multinucleate",systemic,liver,pathology microscopic,liver,2,149.999,lel,mg/kg/day,149.999,'='
3,1,"hepatocyte, multinucleate",systemic,liver,pathology microscopic,liver,2,149.999,nel,mg/kg/day,149.999,<
4,4,body weight gain,systemic,body weight,in life observation,systemic/in life observation,5,56.400,loael,mg/kg/day,56.400,'='


In [5]:
Y.pod_type.value_counts()

noael    14
loael    13
lel      12
nel      11
Name: pod_type, dtype: int64

In [6]:
Y.groupby(['endpoint_category','endpoint_type']).aggregate(dict(pod_type=len))

pod_type
endpoint_category endpoint_type                     
developmental     developmental landmark           7
reproductive      offspring survival early         4
                  reproductive performance         4
systemic          in life observation              7
                  organ weight                     8
                  pathology gross                  4
                  pathology microscopic           16

In [40]:
DB.toxrefdb2.distinct('pods.endpoint_category')

[u'systemic', u'cholinesterase', u'developmental', u'reproductive']

In [13]:
X=DB.toxrefdb2.find_one({'bmds':{'$exists':1},'pods.endpoint_category':u'developmental'})
#X=DB.toxrefdb2.find_one()
Y = pd.DataFrame(X['bmds'])
print X['preferred_name']
Y.head()

Anilazine


,AIC,BMD,BMDL,BMDU,CSF,Chi2,bmr,bmr_type,dataset_id,df,...,model_version,pvalue1,pvalue2,pvalue3,pvalue4,recommended,recommended_variable,residual_of_interest,row_names,warnings
0,315.7288,322.712,210.6380,722.626,-999.0,1.532,1.0,sd,1863_109_28824_F,2.0,...,1.11,<0.0001,0.009946999999999999,0.5218,0.4649,0,AIC,-1,6321,None
1,315.7288,322.712,210.6380,722.626,-999.0,1.532,1.0,sd,1863_109_28824_F,2.0,...,1.11,<0.0001,0.009946999999999999,0.5218,0.4649,0,None,-1,6322,None
2,317.8201,296.183,90.9221,748.435,-999.0,1.623,1.0,sd,1863_109_28824_F,1.0,...,1.11,<0.0001,0.009946999999999999,0.5218,0.2026,0,None,-1.098,6323,None
3,319.816,324.062,91.0256,747.243,-999.0,1.619,1.0,sd,1863_109_28824_F,0.0,...,1.11,<0.0001,0.009946999999999999,0.5218,-999,0,None,-0.9936,6324,None
4,319.815699,323.788,-999.0000,-999.000,-999.0,1.61897,1.0,sd,1863_109_28824_F,0.0,...,2.18,<0.0001,0.009946999999999999,0.5218,-999,0,None,-0.995,6325,BMDL computation failed.


In [14]:
Y2 = pd.DataFrame(X['pods']).sort_values('pod_value')
Y2.query("pod_type=='loael'").head(10)

,dose_level,effect_desc,endpoint_category,endpoint_target,endpoint_type,group_name,max_dose_level,mg_kg_day_value,pod_type,pod_unit,pod_value,qualifier
35,2,mortality,systemic,mortality,in life observation,systemic/in life observation,3,40.0,loael,mg/kg/day,40.0,'='
33,2,body weight gain,systemic,body weight,in life observation,systemic/in life observation,3,40.0,loael,mg/kg/day,40.0,'='
98,1,leukocyte (wbc),systemic,leukocyte (wbc) count differential,hematology,systemic/hematology,3,45.9,loael,mg/kg/day,45.9,'='
64,3,unossified,developmental,bone,developmental malformation,developmental,3,75.0,loael,mg/kg/day,75.0,'='
41,3,postimplantation loss,reproductive,postimplantation loss,reproductive performance,reproductive,3,75.0,loael,mg/kg/day,75.0,'='
37,3,live fetuses,reproductive,live fetuses,offspring survival early,reproductive,3,75.0,loael,mg/kg/day,75.0,'='
17,3,relative to body weight,systemic,testes,organ weight,testes,3,600.0,loael,mg/kg/day,600.0,>
106,3,acanthosis,systemic,skin,pathology microscopic,skin,3,600.0,loael,mg/kg/day,600.0,>
108,3,hyperkeratosis,systemic,skin,pathology microscopic,skin,3,600.0,loael,mg/kg/day,600.0,>
102,3,relative to body weight,systemic,heart,organ weight,heart,3,600.0,loael,mg/kg/day,600.0,>


# Aggregating the LOAEL / LEL

In [15]:
Y2.groupby(['endpoint_category','pod_type']).aggregate(dict(pod_value=min))

pod_value
endpoint_category pod_type           
developmental     lel            75.0
                  loael          75.0
                  nel            40.0
                  noael          40.0
reproductive      lel            75.0
                  loael          75.0
                  nel            40.0
                  noael          40.0
systemic          lel             0.0
                  loael          40.0
                  nel            38.5
                  noael          15.0

In [16]:
Y.columns

Index([u'AIC', u'BMD', u'BMDL', u'BMDU', u'CSF', u'Chi2', u'bmr', u'bmr_type',
       u'dataset_id', u'df', u'doses_dropped', u'endpoint_category',
       u'endpoint_target', u'endpoint_type', u'logic_bin', u'logic_cautions',
       u'logic_failures', u'logic_warnings', u'model_name', u'model_version',
       u'pvalue1', u'pvalue2', u'pvalue3', u'pvalue4', u'recommended',
       u'recommended_variable', u'residual_of_interest', u'row_names',
       u'warnings'],
      dtype='object')

In [17]:
Y.head().T

,0,1,2,3,4
AIC,315.7288,315.7288,317.8201,319.816,319.815699
BMD,322.712,322.712,296.183,324.062,323.788
BMDL,210.638,210.638,90.9221,91.0256,-999
BMDU,722.626,722.626,748.435,747.243,-999
CSF,-999,-999,-999,-999,-999
Chi2,1.532,1.532,1.623,1.619,1.61897
bmr,1,1,1,1,1
bmr_type,sd,sd,sd,sd,sd
dataset_id,1863_109_28824_F,1863_109_28824_F,1863_109_28824_F,1863_109_28824_F,1863_109_28824_F
df,2,2,1,0,0


In [18]:
Y[[u'AIC', u'BMD', u'BMDL', u'BMDU', u'CSF', u'bmr', u'bmr_type','model_name',
    u'endpoint_category',
    u'endpoint_target', u'endpoint_type']].sort_values('BMD')

,AIC,BMD,BMDL,BMDU,CSF,bmr,bmr_type,model_name,endpoint_category,endpoint_target,endpoint_type
69,121.011771,-9.999000e+03,234.798,-9.990000e+02,-999.0,1.0,sd,Polynomial-2,systemic,leukocyte (wbc) count differential,hematology
32,487.462604,-9.999000e+03,2376.530,3.600000e+09,-999.0,1.0,sd,Linear,systemic,lactic acid dehydrogenase (ldh),clinical chemistry
33,487.462603,-9.999000e+03,1237.420,3.600000e+09,-999.0,1.0,sd,Polynomial-2,systemic,lactic acid dehydrogenase (ldh),clinical chemistry
34,487.462603,-9.999000e+03,1074.440,-9.990000e+02,-999.0,1.0,sd,Polynomial-3,systemic,lactic acid dehydrogenase (ldh),clinical chemistry
106,-113.110281,-9.999000e+03,-999.000,-9.990000e+02,-999.0,10.0,rd,Polynomial-3,systemic,body weight,in life observation
105,-113.110281,-9.999000e+03,-999.000,-9.990000e+02,-999.0,10.0,rd,Polynomial-2,systemic,body weight,in life observation
59,69.87371,-9.999000e+03,2302.820,3.600000e+09,-999.0,1.0,sd,Linear,systemic,creatinine,clinical chemistry
60,69.87371,-9.999000e+03,1282.230,-9.990000e+02,-999.0,1.0,sd,Polynomial-2,systemic,creatinine,clinical chemistry
104,-113.110281,-9.999000e+03,-999.000,-9.990000e+02,-999.0,10.0,rd,Linear,systemic,body weight,in life observation
78,157.643841,-9.999000e+03,1538.750,-9.990000e+02,-999.0,1.0,sd,Polynomial-2,systemic,leukocyte (wbc) count differential,hematology


In [19]:
Y.groupby(['endpoint_category','endpoint_type','endpoint_target','bmr_type','model_name','bmr']).aggregate(dict(BMD=min))

BMD
endpoint_category endpoint_type              endpoint_target                bmr_type model_name       bmr             
developmental     developmental malformation bone                           bmr      Dichotomous-Hill 5.0     0.123889
                                                                                                      10.0         NaN
                                                                                     Gamma            5.0    15.706700
                                                                                                      10.0   32.262900
                                                                                     LogLogistic      5.0    14.806600
                                                                                                      10.0   31.258500
                                                                                     LogProbit        5.0    33.715400
                                                                                                      10.0   48.485100
                                                                                     Logistic         5.0    27.547200
                                                                                                      10.0   44.175800
                                                                                     Multistage-2     5.0    15.706700
                                                                                                      10.0   32.262900
                                                                                     Multistage-3     5.0    15.706700
                                                                                                      10.0   32.262900
                                                                                     Probit           5.0    25.943700
                                                                                                      10.0   42.858200
                                                                                     Quantal linear   5.0    15.706700
                                                                                                      10.0   32.262900
                                                                                     Weibull          5.0    15.706600
                                                                                                      10.0   32.262600
systemic          clinical chemistry         alkaline phosphatase (alp/alk) sd       Exponential-M2   1.0   207.779000
                                                                                     Exponential-M3   1.0   207.779000
                                                                                     Exponential-M4   1.0   137.179000
                                                                                     Exponential-M5   1.0   143.970000
                                                                                     Hill             1.0   142.816000
                                                                                     Linear           1.0   181.061000
                                                                                     Polynomial-2     1.0   181.061000
                                                                                     Polynomial-3     1.0   181.061000
                                                                                     Power            1.0   181.061000
                                             creatinine                     sd       Exponential-M2   1.0  -536.678000
...                                                                                                                ...
                  pathology microscopic      stomach                        bmr      Multistage-2     5.0    22.557300
                                                                     

# Aggregating BMD

In [20]:
Y.query("BMD>0").groupby(['endpoint_category','bmr_type','bmr']).aggregate(dict(BMD=min))

BMD
endpoint_category bmr_type bmr               
developmental     bmr      5.0   1.238890e-01
                           10.0  3.125850e+01
systemic          bmr      5.0   1.030920e+01
                           10.0  2.307250e+01
                  rd       10.0  4.314860e-46
                  sd       1.0   1.371790e+02